# Extract results of activation computations for UPP02 to RWCL

Dmitry Portnov, July 2021

## Goals and input data

Extract data from "component_to_cell-up02-1.json", which correspondes to component_to_cell-1.json, attached to https://jira.iterrf.ru:8443/browse/UPP-93, and form a table of values corresponding to RWCL "Radwaste_Checklist_UPP02.xlsx" and "Radwaste_Checklist_UP02_ISS_PCSS.xlsx".

All the files are in the same folder on 10.106.203.11: "d:\dvp\dev\mcnp\upp\wrk\up02".

## Scenario

- Load data from json as map RWCL item -> list of cells in the MCNP model
- Load activation data as map cell -> activation values for 12 day of cooliing



In [1]:
%config Completer.use_jedi = False

In [2]:
import typing as tp
from typing import Dict

In [3]:
import os, sys

In [4]:
try:
    import matplotlib as mpl
except ImportError:
    !pip install matplotlib
    import matplotlib as mpl

import matplotlib.pyplot as plt 
%matplotlib inline

In [5]:
import json
import pathlib

from collections import defaultdict
from pathlib import Path
from pprint import pprint, pformat


In [6]:
import numpy as np
import dotenv
from tqdm import tqdm

In [7]:
try:
    import pandas as pd
except ImportError:
    !pip install pandas
    import pandas as pd

In [8]:
try:
    import openpyxl
except ImportError:
    !pip install openpyxl
    import openpyxl

In [9]:
import mckit as mc
import r2s_rfda as r2s
import r2s_rfda.fetch as r2sft
import r2s_rfda.utils as r2sut
import r2s_rfda.launcher as r2sln

# from r2s_rfda.fetch import load_result_config, load_data, load_result_config
# from r2s_rfda import utils
# from r2s_rfda.launcher import load_config


---*** D:\programs\anaconda3\envs\r2s-0.3\lib\site-packages\mckit\nlopt.dll


In [10]:
# If exists local ".env" file, then load it to environment.
dotenv.load_dotenv();

In [11]:
def all_exist(*paths: Path) -> bool:
    for p in paths: # type: Path
        if not p.exists():
            raise FileNotFoundError(p)
def all_are_files(*paths: Path) -> bool:
    return all(map(Path.is_file, paths))
def all_are_dirs(*paths: Path) -> bool:
    return all(map(Path.is_dir, paths))


In [13]:
ROOT: Path = Path(os.getenv("UP02_ACT_ROOT", r'd:\dvp\dev\mcnp\upp\wrk\up02'))
r2swrk: Path = ROOT  # for UP02 computation were done in the root directory
component_to_cell_json = ROOT / "json-3" / "component_to_cell3.json"
inp_keys_json = ROOT / "json-3" / "inp-keys.json"
inp_iss_keys_json = ROOT / "json-3" / "inp-iss-keys.json"
rwclinput: Path = ROOT / "Radwaste_Checklist_UPP02.xlsx"
rwclinput_iss: Path = ROOT / "Radwaste_Checklist_UP02_ISS_PCSS.xlsx"

all_exist(ROOT, r2swrk, component_to_cell_json, inp_keys_json, inp_iss_keys_json, rwclinput, rwclinput_iss)
assert all_are_dirs(ROOT, r2swrk)
assert all_are_files(pjson, rwclinput, rwclinput_iss)


## Load "component => cells" mapping from JSON

In [14]:
with component_to_cell_json.open() as fid:
    component_to_cell = json.load(fid)
pprint(sorted(component_to_cell.keys()))

['Bioshield plug',
 'Blade 1',
 'Blade 2',
 'Blade 3',
 'Blade 4',
 'Blade 5',
 'Blade 6',
 'Blade 7',
 'Blade 8',
 'Blade 9',
 'Bogies chasis left',
 'Bogies chasis right',
 'Bottom Frame',
 'Bottom Rail',
 'CASK docking connector right',
 'Cask docking connector left',
 'Cask docking rail left',
 'Cask docking rail structure right',
 'DFW Right',
 'DFW_left',
 'DMS SPI Reserve Tank',
 'DSM front',
 'DSM mid',
 'DSM rear',
 'DSM rear2',
 'DSM_mid',
 'Flange 1',
 'Flange ext',
 'HA interspace',
 'HA1',
 'HA2',
 'ISS chasis',
 'Interspace frame',
 'Interspace inner block',
 'Interspace left block',
 'Interspace right block',
 'Interspace zone',
 'Mount bottom',
 'Mount left',
 'Mount right',
 'Mount up',
 'Plate left 1',
 'Plate left 2',
 'Plate left 3',
 'Plate left 4',
 'Plate left 5',
 'Plate left 6',
 'Plate left 7',
 'Plate left 8',
 'Plate right 1',
 'Plate right 2',
 'Plate right 3',
 'Plate right 4',
 'Plate right 5',
 'Plate right 6',
 'Plate right 7',
 'Plate right 8',
 'Rail 

# Load "RWCL key => Component" mapping

In [100]:
with inp_keys_json.open() as fid:
    inp_keys = json.load(fid)
# pprint(sorted(inp_keys.keys()))
pprint(inp_keys)

{'DFW Left': ['DFW_left', 'Mount left', 'Tube left down', 'Tube left up'],
 'DFW Right': ['DFW Right', 'Mount right', 'Tube right down', 'Tube right up'],
 'DSM': ['DSM_mid'],
 'DSM Rails (2 pieces)': ['Top Rail', 'Bottom Rail'],
 'Frame Blade #1': ['Blade 1'],
 'Frame Blade #2': ['Blade 2'],
 'Frame Blade #3-5 (3 pieces)': ['Blade 3', 'Blade 4', 'Blade 5'],
 'Frame Blade #6-10 (5 pieces)': ['Blade 6', 'Blade 7', 'Blade 8', 'Blade 9'],
 'Frame Bottom': ['Bottom Frame'],
 'Frame Top': ['Top Frame'],
 'Generic Screws (50 pieces)': [],
 'PP Generics (pads, skids, gripping - 8 pieces)': [],
 'PP Tubes': [],
 'PP structure': ['Trapezoid', 'Rear Box', 'Flange 1', 'Flange ext'],
 'Rear box shielding (left)': ['Rear left'],
 'Rear box shielding (right)': ['Rear right'],
 'Rear box shielding plates (8 pieces)': [],
 'Rear frame shielding1': ['Shielding segment 1'],
 'Rear frame shielding2': ['Shielding segment 2'],
 'Rear frame shielding3': ['Shielding segment 3'],
 'Rear frame shielding4': ['S

In [20]:
with inp_iss_keys_json.open() as fid:
    inp_iss_keys = json.load(fid)
pprint(inp_iss_keys)

{'Bioshield 1': ['Bioshield plug'],
 'Bioshield 10': ['Bioshield plug'],
 'Bioshield 11': ['Bioshield plug'],
 'Bioshield 12': ['Bioshield plug'],
 'Bioshield 13': ['Bioshield plug'],
 'Bioshield 14': ['Bioshield plug'],
 'Bioshield 15': ['Bioshield plug'],
 'Bioshield 16': ['Bioshield plug'],
 'Bioshield 17': ['Bioshield plug'],
 'Bioshield 18': ['Bioshield plug'],
 'Bioshield 19': ['Bioshield plug'],
 'Bioshield 2': ['Bioshield plug'],
 'Bioshield 20': ['Bioshield plug'],
 'Bioshield 21': ['Bioshield plug'],
 'Bioshield 22': ['Bioshield plug'],
 'Bioshield 23': ['Bioshield plug'],
 'Bioshield 24': ['Bioshield plug'],
 'Bioshield 25': ['Bioshield plug'],
 'Bioshield 26': ['Bioshield plug'],
 'Bioshield 27': ['Bioshield plug'],
 'Bioshield 3': ['Bioshield plug'],
 'Bioshield 4': ['Bioshield plug'],
 'Bioshield 5': ['Bioshield plug'],
 'Bioshield 6': ['Bioshield plug'],
 'Bioshield 7': ['Bioshield plug'],
 'Bioshield 8': ['Bioshield plug'],
 'Bioshield 9': ['Bioshield plug'],
 'Bioshild

Note: JSON doesn't support comments and trailing commas, the JSON texts are fixed

## Load Cell->activation data mapping from R2S working folder

In [21]:
r2s_launch_conf = r2sln.load_config(r2swrk)
r2s_result_conf = r2sft.load_result_config(r2swrk)

In [22]:
time = r2sut.convert_time_literal('12d')
time_labels = list(sorted(r2s_result_conf['gamma'].keys()))
time += time_labels[r2s_launch_conf['zero']]
closest_lab = r2sut.find_closest(time, time_labels)
# closest_lab

In [23]:
r2s_result_conf.keys()

dict_keys(['gamma', 'atoms', 'activity'])

In [24]:
activation_paths = r2s_result_conf["activity"]
activation_paths
act_data = r2sft.load_data(r2swrk / activation_paths[closest_lab])

In [25]:
# Estimate correction factor for tqdm progressbar total value.
its=80625951000
its/act_data._data.nnz

6734.679317774382

In [26]:
tqdm_correction = 6400

In [27]:
def load_cell_activity(cell_excel_path: Path):
    if cell_excel_path.exists():
        cell_activity_df = pd.read_excel(cell_excel_path, sheet_name="activity", index_col=0)
    else:
        cell_activity = defaultdict(float)
        with tqdm(total=act_data._data.nnz * tqdm_correction) as pbar:
            for cnt, ((g, c, i, j, k), act) in enumerate(act_data.iter_nonzero()):
                cell_activity[c] += act
                #print(act_data.gbins[g], c, i, j, k, act)
                #i += 1
                #if i == 1000:
                #    break
                if cnt % 1000 == 0:
                    pbar.update(cnt)
        cell_excel_path.parent.mkdir(parents=True, exist_ok=True)
        index = np.array(sorted(cell_activity.keys()), dtype=np.int32)
        data  = np.fromiter((cell_activity[k] for k in index), dtype=float)
        cell_activity_df = pd.DataFrame(data, columns=["activity"], index=index)
        with pd.ExcelWriter(cell_excel_path, engine="openpyxl") as excel:
            cell_activity_df.to_excel(excel, sheet_name="activity")
    return cell_activity_df
    

In [146]:
def load_cell_nuclide_activity(cell_excel_path: Path, act_data, nuclide: str):
    label = nuclide + " activity"
    loaded_from_cache = False
    if cell_excel_path.exists():
        try:
            cell_activity_df = pd.read_excel(cell_excel_path, sheet_name=label, index_col=0)
            loaded_from_cache = True
        except ValueError:
            loaded_from_cache = False
    
    if not loaded_from_cache:
        nuclide_idx = np.searchsorted(act_data.gbins, nuclide)
        assert act_data.gbins[nuclide_idx] == nuclide
        cell_activity = defaultdict(float)
        with tqdm(total=act_data._data.nnz * tqdm_correction) as pbar:
            for cnt, ((g, c, i, j, k), act) in enumerate(act_data.iter_nonzero()):
                if nuclide_idx == g:
                    cell_activity[c] += act
                if cnt % 1000 == 0:
                    pbar.update(cnt)
        cell_excel_path.parent.mkdir(parents=True, exist_ok=True)
        index = np.array(sorted(cell_activity.keys()), dtype=np.int32)
        data  = np.fromiter((cell_activity[k] for k in index), dtype=float)
        cell_activity_df = pd.DataFrame(data, columns=["activity"], index=index)
        with pd.ExcelWriter(cell_excel_path, engine="openpyxl", mode="a") as excel:
            cell_activity_df.to_excel(excel, sheet_name=label)
    return cell_activity_df
    

In [147]:
cell_activity_df = load_cell_activity(Path.cwd().parent.parent / "wrk/up02/cell_data.xlsx")
cell_activity_df

,activity
320000,4.184432e+13
320001,1.165345e+08
320002,9.482609e+08
320003,5.584514e+07
320004,9.053100e+08
...,...
322492,1.141749e+05
322493,1.582895e+08
322494,2.295881e+08
322755,2.800837e+08


In [148]:
cell_h3_activity_df = load_cell_nuclide_activity(Path.cwd().parent.parent / "wrk/up02/cell_h3_activity.xlsx", act_data, "H3")
cell_h3_activity_df

 94%|█████████████████████████████████████████████████████████████████▍    | 71658406000/76619251200 [12:20<00:51, 96784714.71it/s]


,H3 activity
320000,1.084448e+09
320001,1.437633e+02
320002,6.504507e+03
320003,1.102028e+03
320004,9.091456e+03
...,...
322492,2.132030e-02
322493,2.969989e+02
322494,5.624568e+02
322755,6.160860e+07


In [29]:
cell_activity_df.loc[320000]["activity"]

41844317016729.17

# In vessel RWCL

In [41]:
rwclinput_df = pd.read_excel(rwclinput, sheet_name="UPP02", header=0, usecols=[2,3], index_col=0).iloc[3:]  # Yes, UPP02 for UPP08
rwclinput_df

,Mass of component (see 4.3)
Equipment part (see 4.1),
DFW Left,1106 kg
DFW Right,1043 kg
DSM,3665 kg
Side plate #1-18 (18 pieces),27 kg each
Frame Top,67 kg
Frame Bottom,106 kg
Frame Blade #1,21 kg
Frame Blade #2,21 kg
Frame Blade #3-5 (3 pieces),22 kg each


In [44]:
mass_column = rwclinput_df.columns[0]

In [45]:
in_vessel_components = rwclinput_df.copy()

In [57]:
in_vessel_components["Activity, Bq"] = np.zeros(len(in_vessel_components.index), dtype=float)

In [150]:
in_vessel_components["H3 Activity, Bq"] = np.zeros(len(in_vessel_components.index), dtype=float)

In [120]:
def collect_cells_map(rwcl_key, inp_key_map, component_to_cell_map) -> tp.Optional[Dict[int, float]]:
    keys_in_cell_map = inp_key_map[rwcl_key]
    if keys_in_cell_map:
        cell_fraction_map = defaultdict(float)
        for k in keys_in_cell_map:
            cells_fraction_map_for_key = component_to_cell_map[k]
            if cells_fraction_map_for_key:
                for k, v in cells_fraction_map_for_key.items():
                    cell_fraction_map[int(k)] += v
        for k, v in cell_fraction_map.items():
            if 1.0 < v:
                print(f"Warning: ramp down fraction owverflow in cell {k}: {v:.3g}")
                cell_fraction_map[k] = 1.0
        return cell_fraction_map
    else:
        return None                

In [121]:
def compute_activity(cell_fraction_map: Dict[int, float], cell_activity: pd.DataFrame) -> float:
    
    def mapper(pair: tp.Tuple[int, float]) -> float:
        cell, fraction = pair
        assert isinstance(cell, int), f"Integer cell number is expected, {cell} is found"
        try:
            result = cell_activity.loc[cell].activity * fraction
        except KeyError:
            print(f"Warning: cannot find cell {cell} in cell activity data")
            result = 0.0
        return result
    
    return sum(map(mapper, cell_fraction_map.items()))

In [122]:
for rwclid in in_vessel_components.index:
    cell_fraction_map = collect_cells_map(rwclid, inp_keys, component_to_cell)
    if cell_fraction_map is not None:
        activity = compute_activity(cell_fraction_map, cell_activity_df)
        in_vessel_components.loc[rwclid, ["Activity, Bq"]] = activity
    else:
        print(f"Warning: cannot find activity for RWCL component {rwclid}")

In [159]:
cell_h3_activity_df.rename(columns={"H3 activity":"activity"}, inplace=True)

In [160]:
for rwclid in in_vessel_components.index:
    cell_fraction_map = collect_cells_map(rwclid, inp_keys, component_to_cell)
    if cell_fraction_map is not None:
        activity = compute_activity(cell_fraction_map, cell_h3_activity_df)
        in_vessel_components.loc[rwclid, ["H3 Activity, Bq"]] = activity
    else:
        print(f"Warning: cannot find activity for RWCL component {rwclid}")

In [161]:
in_vessel_components

,Mass of component (see 4.3),"Activity, Bq","H3 Activity, Bq"
Equipment part (see 4.1),,,
DFW Left,1106 kg,2.322408e+15,7.356980e+10
DFW Right,1043 kg,2.135422e+15,6.895064e+10
DSM,3665 kg,9.496197e+13,1.618481e+09
Side plate #1-18 (18 pieces),27 kg each,3.857135e+11,1.091279e+07
Frame Top,67 kg,6.123427e+10,3.107470e+07
Frame Bottom,106 kg,1.592735e+11,2.855502e+07
Frame Blade #1,21 kg,3.369719e+09,7.113522e+05
Frame Blade #2,21 kg,2.873788e+09,4.463410e+05
Frame Blade #3-5 (3 pieces),22 kg each,4.303904e+09,5.940768e+06


In [162]:
ivc1 = in_vessel_components.copy()

In [168]:
ivc1["Mass, kg"] = np.fromiter(map(lambda x: float(x.split()[0]), ivc1[mass_column]), dtype=float)

In [174]:
ivc1["Unit Activity, Bq/kg"] = ivc1["Activity, Bq"] / ivc1["Mass, kg"]

In [175]:
ivc1["Unit H3 Activity, Bq/kg"] = ivc1["H3 Activity, Bq"] / ivc1["Mass, kg"]

In [176]:
ivc1

,"Activity, Bq","H3 Activity, Bq","Mass, kg","Unit Activity, Bq/kg","Unit H3 Activity, Bq/kg"
Equipment part (see 4.1),,,,,
DFW Left,2.322408e+15,7.356980e+10,1106.0,2.099826e+12,6.651881e+07
DFW Right,2.135422e+15,6.895064e+10,1043.0,2.047385e+12,6.610799e+07
DSM,9.496197e+13,1.618481e+09,3665.0,2.591050e+10,4.416046e+05
Side plate #1-18 (18 pieces),3.857135e+11,1.091279e+07,27.0,1.428568e+10,4.041775e+05
Frame Top,6.123427e+10,3.107470e+07,67.0,9.139443e+08,4.638015e+05
Frame Bottom,1.592735e+11,2.855502e+07,106.0,1.502580e+09,2.693870e+05
Frame Blade #1,3.369719e+09,7.113522e+05,21.0,1.604628e+08,3.387391e+04
Frame Blade #2,2.873788e+09,4.463410e+05,21.0,1.368471e+08,2.125433e+04
Frame Blade #3-5 (3 pieces),4.303904e+09,5.940768e+06,22.0,1.956320e+08,2.700349e+05


In [216]:
def create_msg(row):
#     print("Row:", row)
    activity = row["Activity, Bq"]
    unit_activity = row["Unit Activity, Bq/kg"]
    h3_activity = row["H3 Activity, Bq"]
    unit_h3_activity = row["Unit H3 Activity, Bq/kg"]
    if activity > 0.0:
        return f"Activity {activity:.2g} Bq\n({unit_activity:.2g}, Bq/kg),\nH3 Activity {h3_activity:.2g} Bq\n({unit_h3_activity:.2g}, Bq/kg)"
    else:
        return ""



## Interpolate missed values in in-vessel components (ivc1).

"PP tubes" and "PP Generics..." unit values should me the same as for "PP structure", totals - proportional to masses.

"Rear box shielding plates (8 pieces)" - as for "Rear frame shielding9"

In [3]:
def update_missed_components(df, refkey, missed_keys):
    unit_activity = df.loc[refkey, ["Unit Activity, Bq/kg"]].values[0]
    unit_h3_activity = df.loc[refkey, ["Unit H3 Activity, Bq/kg"]].values[0]
    df.loc[missed_keys, ["Unit Activity, Bq/kg"]] =  unit_activity
    df.loc[missed_keys, ["Unit H3 Activity, Bq/kg"]] = unit_h3_activity
    masses = df.loc[missed_keys, ["Mass, kg"]].values
    df.loc[missed_keys, ["Activity, Bq"]] = unit_activity * masses
    df.loc[missed_keys, ["H3 Activity, Bq"]] = unit_h3_activity * masses
    return df.loc[missed_keys]

In [344]:
def update_missed_pp_components():
    missed_keys = ["PP Tubes", "PP Generics (pads, skids, gripping - 8 pieces)", "Generic Screws (50 pieces)"]
    refkey = "PP structure"
    return update_missed_components(ivc1, refkey, missed_keys)

update_missed_pp_components()

,"Mass, kg","Activity, Bq","Unit Activity, Bq/kg","H3 Activity, Bq","Unit H3 Activity, Bq/kg",Radiological data
Equipment part (see 4.1),,,,,,
PP Tubes,119.0,4.050791e+12,3.404026e+10,3.172920e+07,266631.938896,"Activity 4.1e+12 Bq\n(3.4e+10, Bq/kg),\nH3 Act..."
"PP Generics (pads, skids, gripping - 8 pieces)",199.0,6.774012e+12,3.404026e+10,5.305976e+07,266631.938896,"Activity 6.8e+12 Bq\n(3.4e+10, Bq/kg),\nH3 Act..."
Generic Screws (50 pieces),45.0,1.531812e+12,3.404026e+10,1.199844e+07,266631.938896,"Activity 1.5e+12 Bq\n(3.4e+10, Bq/kg),\nH3 Act..."


In [345]:
def update_missed_rb_components():
    missed_keys = ["Rear box shielding plates (8 pieces)", "Rear frame shielding9"]
    refkey = "Rear frame shielding8"
    return update_missed_components(ivc1, refkey, missed_keys)

update_missed_rb_components()

,"Mass, kg","Activity, Bq","Unit Activity, Bq/kg","H3 Activity, Bq","Unit H3 Activity, Bq/kg",Radiological data
Equipment part (see 4.1),,,,,,
Rear box shielding plates (8 pieces),315.0,4.384030e+08,1.391756e+06,2.210589e+07,70177.429273,"Activity 4.4e+08 Bq\n(1.4e+06, Bq/kg),\nH3 Act..."
Rear frame shielding9,194.0,2.700006e+08,1.391756e+06,1.361442e+07,70177.429273,"Activity 2.7e+08 Bq\n(1.4e+06, Bq/kg),\nH3 Act..."


In [346]:
# ivc1=ivc1.reindex(columns=["Mass, kg", "Activity, Bq", "Unit Activity, Bq/kg", "H3 Activity, Bq", "Unit H3 Activity, Bq/kg"])
ivc1["Radiological data"] = ivc1.apply(create_msg, axis=1)

In [347]:
ivc1

,"Mass, kg","Activity, Bq","Unit Activity, Bq/kg","H3 Activity, Bq","Unit H3 Activity, Bq/kg",Radiological data
Equipment part (see 4.1),,,,,,
DFW Left,1106.0,2.322408e+15,2.099826e+12,7.356980e+10,6.651881e+07,"Activity 2.3e+15 Bq\n(2.1e+12, Bq/kg),\nH3 Act..."
DFW Right,1043.0,2.135422e+15,2.047385e+12,6.895064e+10,6.610799e+07,"Activity 2.1e+15 Bq\n(2e+12, Bq/kg),\nH3 Activ..."
DSM,3665.0,9.496197e+13,2.591050e+10,1.618481e+09,4.416046e+05,"Activity 9.5e+13 Bq\n(2.6e+10, Bq/kg),\nH3 Act..."
Side plate #1-18 (18 pieces),27.0,3.857135e+11,1.428568e+10,1.091279e+07,4.041775e+05,"Activity 3.9e+11 Bq\n(1.4e+10, Bq/kg),\nH3 Act..."
Frame Top,67.0,6.123427e+10,9.139443e+08,3.107470e+07,4.638015e+05,"Activity 6.1e+10 Bq\n(9.1e+08, Bq/kg),\nH3 Act..."
Frame Bottom,106.0,1.592735e+11,1.502580e+09,2.855502e+07,2.693870e+05,"Activity 1.6e+11 Bq\n(1.5e+09, Bq/kg),\nH3 Act..."
Frame Blade #1,21.0,3.369719e+09,1.604628e+08,7.113522e+05,3.387391e+04,"Activity 3.4e+09 Bq\n(1.6e+08, Bq/kg),\nH3 Act..."
Frame Blade #2,21.0,2.873788e+09,1.368471e+08,4.463410e+05,2.125433e+04,"Activity 2.9e+09 Bq\n(1.4e+08, Bq/kg),\nH3 Act..."
Frame Blade #3-5 (3 pieces),22.0,4.303904e+09,1.956320e+08,5.940768e+06,2.700349e+05,"Activity 4.3e+09 Bq\n(2e+08, Bq/kg),\nH3 Activ..."


In [348]:
with pd.ExcelWriter(ROOT / "in-vessel.xlsx", engine="openpyxl", mode="w") as excel:
    ivc1.to_excel(excel, sheet_name="result")

# ISS components

In [273]:
rwclinput_iss_df = pd.read_excel(rwclinput_iss, sheet_name="UPP02", header=0, usecols=[2,3], index_col=0).iloc[3:]  
rwclinput_iss_df.rename(columns={mass_column: "Mass, kg"}, inplace=True)
rwclinput_iss_df

,"Mass, kg"
Equipment part (see 4.1),
PC Locking System,118.0
PC Bogies,1812.0
PC Frame,212.0
PC Chasis,1224.0
IS Diagnostic sheet,441.0
IS Frame,407.0
IS Chasis,4800.0
IS Bogies,2420.0
IS Locking System,82.0


In [274]:
for i in rwclinput_iss_df.index:
    print(i, ":", inp_iss_keys[i])

PC Locking System : []
PC Bogies : []
PC Frame : []
PC Chasis : []
IS Diagnostic sheet : ['ISS chasis']
IS Frame : ['Support frame', 'Interspace frame']
IS Chasis : ['ISS chasis']
IS Bogies : ['Bogies chasis left', 'Bogies chasis right']
IS Locking System : []
IS Chasis Shielding Cube : ['ISS chasis']
IS Cable tubes : []
IS Add Protection : []
IS Protection Frame : []
IS Walkways : []
Bioshild_front_1 : ['Interspace left block']
Bioshild_front_2 : ['Interspace inner block']
Bioshild_front_3 : ['Interspace right block']
Bioshield 1 : ['Bioshield plug']
Bioshield 2 : ['Bioshield plug']
Bioshield 3 : ['Bioshield plug']
Bioshield 4 : ['Bioshield plug']
Bioshield 5 : ['Bioshield plug']
Bioshield 6 : ['Bioshield plug']
Bioshield 7 : ['Bioshield plug']
Bioshield 8 : ['Bioshield plug']
Bioshield 9 : ['Bioshield plug']
Bioshield 10 : ['Bioshield plug']
Bioshield 11 : ['Bioshield plug']
Bioshield 12 : ['Bioshield plug']
Bioshield 13 : ['Bioshield plug']
Bioshield 14 : ['Bioshield plug']
Bioshiel

In [275]:
iss_components = rwclinput_iss_df.copy()

In [276]:
iss_components["Activity, Bq"] = np.zeros(len(iss_components.index))
iss_components["Unit Activity, Bq/kg"] = np.zeros(len(iss_components.index))
iss_components["H3 Activity, Bq"] = np.zeros(len(iss_components.index))
iss_components["Unit H3 Activity, Bq/kg"] = np.zeros(len(iss_components.index))

In [277]:
for rwclid in iss_components.index:
    cell_fraction_map = collect_cells_map(rwclid, inp_iss_keys, component_to_cell)
    if cell_fraction_map is not None:
        activity = compute_activity(cell_fraction_map, cell_activity_df)
        iss_components.loc[rwclid, ["Activity, Bq"]] = activity
    else:
        print(f"Warning: cannot find activity for RWCL component {rwclid}")

In [278]:
for rwclid in iss_components.index:
    cell_fraction_map = collect_cells_map(rwclid, inp_iss_keys, component_to_cell)
    if cell_fraction_map is not None:
        activity = compute_activity(cell_fraction_map, cell_h3_activity_df)
        iss_components.loc[rwclid, ["H3 Activity, Bq"]] = activity
    else:
        print(f"Warning: cannot find activity for RWCL component {rwclid}")

In [279]:
iss_components

,"Mass, kg","Activity, Bq","Unit Activity, Bq/kg","H3 Activity, Bq","Unit H3 Activity, Bq/kg"
Equipment part (see 4.1),,,,,
PC Locking System,118.0,0.000000e+00,0.0,0.000000e+00,0.0
PC Bogies,1812.0,0.000000e+00,0.0,0.000000e+00,0.0
PC Frame,212.0,0.000000e+00,0.0,0.000000e+00,0.0
PC Chasis,1224.0,0.000000e+00,0.0,0.000000e+00,0.0
IS Diagnostic sheet,441.0,3.777029e+09,0.0,6.755402e+06,0.0
IS Frame,407.0,5.315905e+08,0.0,6.912833e+01,0.0
IS Chasis,4800.0,3.777029e+09,0.0,6.755402e+06,0.0
IS Bogies,2420.0,6.314367e+08,0.0,1.912921e+03,0.0
IS Locking System,82.0,0.000000e+00,0.0,0.000000e+00,0.0


## Set "Unit" values

In [280]:
iss_components["Unit Activity, Bq/kg"] = iss_components["Activity, Bq"] / iss_components["Mass, kg"]
iss_components["Unit H3 Activity, Bq/kg"] = iss_components["H3 Activity, Bq"] / iss_components["Mass, kg"]
iss_components

,"Mass, kg","Activity, Bq","Unit Activity, Bq/kg","H3 Activity, Bq","Unit H3 Activity, Bq/kg"
Equipment part (see 4.1),,,,,
PC Locking System,118.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
PC Bogies,1812.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
PC Frame,212.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
PC Chasis,1224.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
IS Diagnostic sheet,441.0,3.777029e+09,8.564692e+06,6.755402e+06,1.531837e+04
IS Frame,407.0,5.315905e+08,1.306119e+06,6.912833e+01,1.698485e-01
IS Chasis,4800.0,3.777029e+09,7.868811e+05,6.755402e+06,1.407375e+03
IS Bogies,2420.0,6.314367e+08,2.609243e+05,1.912921e+03,7.904632e-01
IS Locking System,82.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


## Correct "Bioshield #" pieces

All 27 components "Bioshield \<number\>" are presented  as a single component in our model, so the totals should be distributed proportionaly to masses
Unit values should the same for all these components.

In [281]:
bioshield_components = list(f"Bioshield {i}" for i in range(1, 28))

In [282]:
iss_components.loc[bioshield_components]

,"Mass, kg","Activity, Bq","Unit Activity, Bq/kg","H3 Activity, Bq","Unit H3 Activity, Bq/kg"
Equipment part (see 4.1),,,,,
Bioshield 1,598.0,3.283149e+09,5.490215e+06,2.456975e+08,4.108653e+05
Bioshield 2,621.0,3.283149e+09,5.286874e+06,2.456975e+08,3.956481e+05
Bioshield 3,903.0,3.283149e+09,3.635824e+06,2.456975e+08,2.720902e+05
Bioshield 4,219.0,3.283149e+09,1.499155e+07,2.456975e+08,1.121906e+06
Bioshield 5,850.0,3.283149e+09,3.862528e+06,2.456975e+08,2.890558e+05
Bioshield 6,975.0,3.283149e+09,3.367332e+06,2.456975e+08,2.519974e+05
Bioshield 7,410.0,3.283149e+09,8.007680e+06,2.456975e+08,5.992621e+05
Bioshield 8,510.0,3.283149e+09,6.437546e+06,2.456975e+08,4.817597e+05
Bioshield 9,742.0,3.283149e+09,4.424729e+06,2.456975e+08,3.311287e+05


In [283]:
bioshield_mass = iss_components.loc[bioshield_components]["Mass, kg"].sum()
bioshield_mass

19535.0

In [284]:
bioshield_unit_activity = iss_components.loc[bioshield_components[0], ["Activity, Bq"]].values[0] / bioshield_mass
bioshield_unit_activity

168064.9415361145

In [285]:
bioshield_unit_h3_activity = iss_components.loc[bioshield_components[0], ["H3 Activity, Bq"]].values[0] / bioshield_mass
bioshield_unit_h3_activity

12577.295683016637

In [287]:
iss_components_bak = iss_components.copy()

In [288]:
iss_components.loc[bioshield_components, ["Unit Activity, Bq/kg"]] = bioshield_unit_activity
iss_components.loc[bioshield_components, ["Unit H3 Activity, Bq/kg"]] = bioshield_unit_h3_activity
iss_components

,"Mass, kg","Activity, Bq","Unit Activity, Bq/kg","H3 Activity, Bq","Unit H3 Activity, Bq/kg"
Equipment part (see 4.1),,,,,
PC Locking System,118.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
PC Bogies,1812.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
PC Frame,212.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
PC Chasis,1224.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
IS Diagnostic sheet,441.0,3.777029e+09,8.564692e+06,6.755402e+06,15318.372240
IS Frame,407.0,5.315905e+08,1.306119e+06,6.912833e+01,0.169848
IS Chasis,4800.0,3.777029e+09,7.868811e+05,6.755402e+06,1407.375450
IS Bogies,2420.0,6.314367e+08,2.609243e+05,1.912921e+03,0.790463
IS Locking System,82.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000


In [299]:
# set total activity proportional to mass
masses = iss_components.loc[bioshield_components, ["Mass, kg"]].values
iss_components.loc[bioshield_components, ["Activity, Bq"]] =  iss_components.loc[bioshield_components, ["Unit Activity, Bq/kg"]].values * masses
iss_components.loc[bioshield_components, ["H3 Activity, Bq"]] =  iss_components.loc[bioshield_components, ["Unit H3 Activity, Bq/kg"]].values * masses
iss_components

,"Mass, kg","Activity, Bq","Unit Activity, Bq/kg","H3 Activity, Bq","Unit H3 Activity, Bq/kg"
Equipment part (see 4.1),,,,,
PC Locking System,118.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
PC Bogies,1812.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
PC Frame,212.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
PC Chasis,1224.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
IS Diagnostic sheet,441.0,3.777029e+09,8.564692e+06,6.755402e+06,15318.372240
IS Frame,407.0,5.315905e+08,1.306119e+06,6.912833e+01,0.169848
IS Chasis,4800.0,3.777029e+09,7.868811e+05,6.755402e+06,1407.375450
IS Bogies,2420.0,6.314367e+08,2.609243e+05,1.912921e+03,0.790463
IS Locking System,82.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000


## Interpolate missed data for ISS

*Note*: port cell data are not computed in this model

In [349]:
def update_missed_iss_components():
    missed_keys = ["IS Locking System", "IS Cable tubes", "IS Add Protection", "IS Protection Frame"]
    refkey = "IS Bogies"
    update_missed_components(iss_components, refkey, missed_keys)
    missed_keys = ["IS Walkways"]
    refkey = "IS Chasis"
    update_missed_components(iss_components, refkey, missed_keys)

update_missed_iss_components()

In [350]:
iss_components["Radiological data"] = iss_components.apply(create_msg, axis=1)
iss_components

,"Mass, kg","Activity, Bq","Unit Activity, Bq/kg","H3 Activity, Bq","Unit H3 Activity, Bq/kg",Radiological data
Equipment part (see 4.1),,,,,,
PC Locking System,118.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,
PC Bogies,1812.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,
PC Frame,212.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,
PC Chasis,1224.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,
IS Diagnostic sheet,441.0,3.777029e+09,8.564692e+06,6.755402e+06,15318.372240,"Activity 3.8e+09 Bq\n(8.6e+06, Bq/kg),\nH3 Act..."
IS Frame,407.0,5.315905e+08,1.306119e+06,6.912833e+01,0.169848,"Activity 5.3e+08 Bq\n(1.3e+06, Bq/kg),\nH3 Act..."
IS Chasis,4800.0,3.777029e+09,7.868811e+05,6.755402e+06,1407.375450,"Activity 3.8e+09 Bq\n(7.9e+05, Bq/kg),\nH3 Act..."
IS Bogies,2420.0,6.314367e+08,2.609243e+05,1.912921e+03,0.790463,"Activity 6.3e+08 Bq\n(2.6e+05, Bq/kg),\nH3 Act..."
IS Locking System,82.0,2.139579e+07,2.609243e+05,6.481798e+01,0.790463,"Activity 2.1e+07 Bq\n(2.6e+05, Bq/kg),\nH3 Act..."


In [351]:
with pd.ExcelWriter(ROOT / "iss.xlsx", engine="openpyxl", mode="w") as excel:
    iss_components.to_excel(excel, sheet_name="result")

In [352]:
ROOT

WindowsPath('d:/dvp/dev/mcnp/upp/wrk/up02')